In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sys
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import pickle
from collections import OrderedDict
import copy
from scipy.sparse import csr_matrix
from scipy import io
import seaborn as sns
import joblib
# from base import *
from joblib import Parallel, delayed
import random
import scipy

In [3]:
MNM_nb_folder = os.path.join('..', '..', '..', 'side_project', 'network_builder')
sys.path.append(MNM_nb_folder)
python_lib_folder = os.path.join('..', '..', 'pylib')
sys.path.append(python_lib_folder)

In [4]:
from MNMAPI import *
from MNM_mcnb import *
from mcDODE import *

In [5]:
data_folder = os.path.join('/home/lemma/Documents/MAC-POSTS/data/input_files_7link_multiclass')

In [6]:
nb = MNM_network_builder()
nb.load_from_folder(data_folder)

In [7]:
observed_link_list = [3, 4, 6]
ml_car = 2
ml_truck = 1
data_dict = dict()
num_interval = nb.config.config_dict['DTA']['max_interval']
true_car_x = np.random.rand(num_interval * len(observed_link_list)) * 100
true_truck_x = np.random.rand(num_interval * len(observed_link_list)) * 10
L_car_one = np.random.randint(2, size = (ml_car, len(observed_link_list)))
L_truck_one = np.random.randint(2, size = (ml_truck, len(observed_link_list)))
L_car = csr_matrix(scipy.linalg.block_diag(*[L_car_one for i in range(num_interval)]))
L_truck = csr_matrix(scipy.linalg.block_diag(*[L_truck_one for i in range(num_interval)]))
m_car = L_car.dot(true_car_x)
m_truck = L_truck.dot(true_truck_x)
data_dict['car_count_agg_L_list'] = [L_car]
data_dict['truck_count_agg_L_list'] = [L_truck]
data_dict['car_link_flow'] = [m_car]
data_dict['truck_link_flow'] = [m_truck]
# data_dict['car_link_tt'] = [m_spd_car]
# data_dict['truck_link_tt'] = [m_spd_truck]

In [8]:
config = dict()
config['use_car_link_flow'] = True
config['use_truck_link_flow'] = False
config['use_car_link_tt'] = False
config['use_truck_link_tt'] = False
config['car_count_agg'] = True
config['truck_count_agg'] = True
config['link_car_flow_weight'] = 1
config['link_truck_flow_weight'] = 1
config['link_tt_weight'] = 1
config['num_data'] = 1
config['observed_links'] = observed_link_list
config['paths_list'] = range(nb.config.config_dict['FIXED']['num_path'])

In [9]:
nb.config

[DTA]
network_name = Snap_graph
unit_time = 5
total_interval = -1
assign_frq = 180
start_assign_interval = 0
max_interval = 10
flow_scalar = 10
num_of_link = 7
num_of_node = 6
num_of_O = 1
num_of_D = 1
OD_pair = 1
adaptive_ratio = 0.5
routing_type = Hybrid

[STAT]
rec_mode = LRn
rec_mode_para = 12
rec_folder = record
rec_volume = 0
volume_load_automatic_rec = 0
volume_record_automatic_rec = 0
rec_tt = 1
tt_load_automatic_rec = 0
tt_record_automatic_rec = 0

[FIXED]
path_file_name = path_table
num_path = 3
choice_portion = Buffer
route_frq = 180

[ADAPTIVE]
route_frq = 180

In [10]:
dode = MCDODE(nb, config)

In [11]:
dode.add_data(data_dict)

In [12]:
(car_flow, truck_flow) = dode.init_path_flow(car_scale = 10, truck_scale = 1)

In [13]:
dode.estimate_path_flow(max_epoch = 10, car_init_scale = 100, truck_init_scale = 10, store_folder = '.')

Start iteration 1530501851.22
Running simulation 1530501851.22
Finish simulation 1530501851.25
Getting DAR 1530501851.25
Creating the coo matrix 1530501851.26
converting the csr 1530501851.26
finish converting 1530501851.26
Evaluating grad 1530501851.26
car link flow 1530501851.26
Getting Loss 1530501851.26
Epoch: 0 Loss: 142.07859148568352 1530501851.26
Start iteration 1530501851.26
Running simulation 1530501851.26
Finish simulation 1530501851.3
Getting DAR 1530501851.3
Creating the coo matrix 1530501851.3
converting the csr 1530501851.3
finish converting 1530501851.3
Evaluating grad 1530501851.3
car link flow 1530501851.3
Getting Loss 1530501851.3
Epoch: 1 Loss: 133.3831062935849 1530501851.31
Start iteration 1530501851.31
Running simulation 1530501851.31
Finish simulation 1530501851.34
Getting DAR 1530501851.34
Creating the coo matrix 1530501851.35
converting the csr 1530501851.35
finish converting 1530501851.35
Evaluating grad 1530501851.35
car link flow 1530501851.35
Getting Loss 

(array([64.64658627, 46.60916825,  4.19372478,  3.35645171, 34.20828135,
        58.24645595, 78.65160324, 62.17462759, 18.45688847, 45.06245272,
        31.1264157 , 74.30924781, 37.81039113, 35.17064768, 84.7312193 ,
        21.10081931, 60.38769706, 39.46148076,  0.93694497,  1.8191899 ,
        84.89529249, 57.5238627 , 30.27235348, 59.527687  , 60.96551992,
        73.35167133, 46.07184872, 63.36892599, 52.47307896, 64.61125576]),
 array([0.74339282, 3.61978535, 6.17018451, 6.37186652, 0.11797884,
        3.71984908, 1.4799902 , 4.57439874, 4.1687106 , 3.0392103 ,
        9.20070639, 0.36711798, 2.80973578, 9.59546815, 8.23172382,
        8.86668511, 0.49368852, 3.70662858, 3.08921281, 0.0485392 ,
        2.06966675, 1.16858713, 8.56799518, 6.52262176, 7.16177778,
        1.92668605, 1.0987428 , 4.56185341, 6.56324267, 1.44802636]))

In [14]:
from scipy.sparse import coo_matrix